## Train data prep

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import random 
import pickle
from tqdm.notebook import tqdm

DATA_DIR = "D:/learning/tf/EMOTION_DETECTION/data/train" # 48x48 px image
CATEGORIES = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

training_data = []

def create_training_data():
    with tqdm(total=len(CATEGORIES)) as pbar_cat:
        for CATEGORY in CATEGORIES:
            path = os.path.join(DATA_DIR, CATEGORY)
            imgs_list = os.listdir(path)
            with tqdm(total=len(imgs_list)) as pbar_img:
                for img_name in os.listdir(path):
                    class_ = CATEGORIES.index(CATEGORY)
                    img_array = cv2.imread(os.path.join(path, img_name), cv2.IMREAD_GRAYSCALE)
                    training_data.append([img_array, class_])
                    pbar_img.update(1)
            pbar_cat.update(1)
            
create_training_data()


random.shuffle(training_data)

IMG_SIZE = 48

X=[]
y=[]

with tqdm(total=len(CATEGORIES)) as pbar_t_data:
    for features, label in training_data:
        X.append(features)
        y.append(label)
        pbar_t_data.update(1)


X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1) #1 for gray scale, 3 for color img0
y = np.array(y)

 # expoet cleaned data

pickle_out=open("X-{}.pickle".format(IMG_SIZE), "wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out=open("y-{}.pickle".format(IMG_SIZE), "wb")
pickle.dump(y, pickle_out)
pickle_out.close()

## TEST data prep for model evaluation

In [ ]:
import random 
import pickle
import os
import cv2 
import numpy as np

TEST_DATA_DIR = "D:/learning/tf/EMOTION_DETECTION/data/test" # 48x48 px image
CATEGORIES = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

testing_data = []

def create_testing_data():
    for CATEGORY in CATEGORIES:
        path = os.path.join(TEST_DATA_DIR, CATEGORY)
        for img_name in os.listdir(path):
            class_ = CATEGORIES.index(CATEGORY)
            img_array = cv2.imread(os.path.join(path, img_name), cv2.IMREAD_GRAYSCALE)
            testing_data.append([img_array, class_])
            
create_testing_data()

random.shuffle(testing_data)

IMG_SIZE = 48

X_test=[]
y_test=[]

for features, label in testing_data:
    X_test.append(features)
    y_test.append(label)
    
X_test = np.array(X_test).reshape(-1, IMG_SIZE, IMG_SIZE, 1) #1 for gray scale, 3 for color img0
y_test = np.array(y_test)


pickle_out=open("X-test-{}.pickle".format(IMG_SIZE), "wb")
pickle.dump(X_test, pickle_out)
pickle_out.close()

pickle_out=open("y-test-{}.pickle".format(IMG_SIZE), "wb")
pickle.dump(y_test, pickle_out)
pickle_out.close()


# predictions

In [ ]:
import tensorflow as tf

MODEL_DIR = "./old_models/"

MODEL_NAME = "emo-analysis-Adam-decay-lr-10-epoch-3-conv-128-nodes-0-dense-1613888403.model"

for model in os.listdir(MODEL_DIR):
    print(model)
    model = tf.keras.models.load_model(MODEL_DIR + model)
    model_eval = model.evaluate(X_test, y_test)
    print(model_eval)

## External image predictions

In [ ]:
import cv2
import tensorflow as tf
import os
import numpy as np
from tqdm.notebook import tqdm, trange

IMG_SIZE = 48

TEST_DIR_PATH = "D:/learning/tf/EMOTION_DETECTION/extdata"

MODEL_DIR = "./old_models/"

#best so far 
## emo-analysis-Adam-decay-lr-3-conv-128-nodes-1-dense-1613837977.model

MODEL_NAME = "emo-analysis-Adam-decay-lr-3-conv-128-nodes-1-dense-1613837977.model"

CATEGORIES = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

def prepare(path):
    img_array = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1) # 1:gray scale 3:color image

model = tf.keras.models.load_model(MODEL_DIR + MODEL_NAME)



for img_path in os.listdir(TEST_DIR_PATH):
    preprd_img = prepare(os.path.join(TEST_DIR_PATH, img_path))
    pred = model.predict([preprd_img])
#     print(np.argmax(pred))
    print("{} - {}".format(os.path.join(TEST_DIR_PATH, img_path), CATEGORIES[np.argmax(pred)]))

## Live Emotion Detection usng OpenCV

In [ ]:
import numpy as np
import cv2
import time as t
import tensorflow as tf

IMG_SIZE = 48

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

cap = cv2.VideoCapture(0)


STRECH = 10

CATEGORIES = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

COLORS = {
     'angry':  (0, 0, 255), 
     'disgust': (0, 255, 255), 
     'fear':    (255,145,0), 
     'happy':   (0, 255, 0), 
     'neutral': (0, 53, 255), 
     'sad':     (255, 26, 0), 
     'surprise':(255, 0, 255)
}

MODEL_DIR = "./best/"
MODEL_NAME = "emo-analysis-Adam-decay-lr-3-conv-128-nodes-1-dense-1613837977.model" # - best so far

# MODEL_DIR = "./models/"
# MODEL_NAME = "emo-analysis-Adam-decay-lr-10-epoch-2-conv-256-nodes-1-dense-1613917992.model"

model = tf.keras.models.load_model(MODEL_DIR + MODEL_NAME)

window_name = 'Image'
font = cv2.FONT_HERSHEY_SIMPLEX
org = (50, 50)
fontScale = 1
color = (255, 0, 0) 
thickness = 2
emotions = []
while 1:
    try:
        ret, img = cap.read()
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        time_str = str(int(t.time()))
        for (x,y,w,h) in faces:
            x-=STRECH
            y-=STRECH
            w+=STRECH*2
            h+=STRECH*2
            roi_gray = gray[y:y+h, x:x+w]
    #         roi_color = img[y:y+h, x:x+w]
            img_array = cv2.resize(roi_gray, (IMG_SIZE, IMG_SIZE))
            img_array = img_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)
            org = (x-10, y-10)
            pred = model.predict([img_array])
            emotion = CATEGORIES[np.argmax(pred)]
            emotions.append(emotion)
            cv2.rectangle(img,(x,y),(x+w,y+h),COLORS[emotion],2)
            img=cv2.putText(img,emotion, org, font,
                           fontScale, COLORS[emotion], thickness, cv2.LINE_AA)

        cv2.imshow('img',img)
        cv2.imwrite(r'./faces/'+'face-'+','.join(emotions)+'.png', img)
        emotions.clear()
    except Exception as e:
        pass
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()